# TCP Server

## Exemplu 1: serverul trimite mesaj la conectarea unui client

In [1]:
import socket

# Adresa IP Server - localhost (daca rulam pe acelasi PC)
HOST = "127.0.0.1"
# Port server. Porturile >1023 sunt neprivilegiate
PORT = 34512

# Crearea unui socket TCP
with socket.socket( socket.AF_INET, socket.SOCK_STREAM ) as tcp_sock:
    # Asignarea unui IP si a unui port pe care serverul sa asculte
    tcp_sock.bind( (HOST, PORT) )
    # Serverul incepe astepte conexiuni pe IP-ul si portul asignat
    tcp_sock.listen()
    # Serverul blocheaza executia si asteapta conectarea unui client
    connection, address = tcp_sock.accept()
    
    # Verificare daca conexiunea a fost realizata
    with connection:
        print( "Info client conectat (IP Client: ", address[0], ", Port client: ", address[1], ")" )
        # Construire mesaj catre client
        final_message = "Salutari, client"
        # Codificare mesaj ca UTF-8
        final_message_bytes = str.encode( final_message )
        # Trimiterea intregului mesaj catre client pe socket
        connection.sendall( final_message_bytes )
    
    # Inchidere socket
    tcp_sock.close()

Info client conectat (IP Client:  127.0.0.1 , Port client:  62218 )


## Exemplu 2: serverul primeste mesaj de la client si raspunde

In [1]:
import socket

# Adresa IP Server - localhost (daca rulam pe acelasi PC)
HOST = "127.0.0.1"
# Port server. Porturile >1023 sunt neprivilegiate
PORT = 34512

# Dimensiunea maxima care va fi primita pe socket de la client
BUFFER_SIZE  = 1024

# Counter care va inregistra numarul de socket-uri deschise catre server
conn_idx = 0

while True:
    with socket.socket( socket.AF_INET, socket.SOCK_STREAM ) as tcp_sock:
        tcp_sock.bind( (HOST, PORT) )
        tcp_sock.listen()
        conn, addr = tcp_sock.accept()
        with conn:
            print( "Info client conectat (IP Client: ", addr[0], ", Port client: ", addr[1], ")" )
            
            data = conn.recv( BUFFER_SIZE )
            
            # Verificare daca au fost primite date de la client
            if( not data ):
                break
            # Decodare mesaj primit de la client 
            decoded_data = data.decode()
            # Spargere string provenit de la client si extragere numarului clientului
            client_number = decoded_data.split(" ")[-1]

            # Construire mesaj catre client
            final_message = "Salut client " + str( client_number ) + ", esti conexiunea numarul " + str( conn_idx )
            # Codificare mesaj ca UTF-8
            final_message_bytes = str.encode( final_message )
            conn_idx += 1
            # Trimiterea intregului mesaj pe socket
            conn.sendall( final_message_bytes )
        
        tcp_sock.close()

Info client conectat (IP Client:  127.0.0.1 , Port client:  62437 )
Info client conectat (IP Client:  127.0.0.1 , Port client:  62438 )
Info client conectat (IP Client:  127.0.0.1 , Port client:  62440 )
Info client conectat (IP Client:  127.0.0.1 , Port client:  62441 )
Info client conectat (IP Client:  127.0.0.1 , Port client:  62442 )
Info client conectat (IP Client:  127.0.0.1 , Port client:  62443 )
Info client conectat (IP Client:  127.0.0.1 , Port client:  62451 )
Info client conectat (IP Client:  127.0.0.1 , Port client:  62452 )
Info client conectat (IP Client:  127.0.0.1 , Port client:  62453 )
Info client conectat (IP Client:  127.0.0.1 , Port client:  62455 )
Info client conectat (IP Client:  127.0.0.1 , Port client:  62456 )
Info client conectat (IP Client:  127.0.0.1 , Port client:  62457 )
Info client conectat (IP Client:  127.0.0.1 , Port client:  62458 )
Info client conectat (IP Client:  127.0.0.1 , Port client:  62460 )
Info client conectat (IP Client:  127.0.0.1 , Po

KeyboardInterrupt: 

## Exemplu 3.1: serverul primeste o imagine de la client si o salveaza intr-un fisier

In [7]:
import socket
import cv2
import numpy as np

HOST = "127.0.0.1"
PORT = 34512

# Dimensiunea imaginii asteptate
BUFFER_SIZE  = 4000
# Deschiderea unui fisier gol unde va fi salvata imaginea
file = open('server_cat.png', 'wb')

with socket.socket( socket.AF_INET, socket.SOCK_STREAM ) as tcp_sock:
    tcp_sock.bind( (HOST, PORT) )
    tcp_sock.listen()
    conn, addr = tcp_sock.accept()
    with conn:
        print( "Info client conectat (IP Client: ", addr[0], ", Port client: ", addr[1], ")" )
        # Primirea imaginii
        data = conn.recv( BUFFER_SIZE )
        if( data ):
            # Scrierea imaginii pe disk
            file.write( data )

    file.close()
    tcp_sock.close()

Info client conectat (IP Client:  127.0.0.1 , Port client:  62082 )


## Exemplu 3.2: serverul primeste o imagine de la client si o afiseaza

In [2]:
import socket
import cv2
import numpy as np

HOST = "127.0.0.1"
PORT = 34512

# Dimensiunea imaginii asteptate
BUFFER_SIZE  = 19200

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as tcp_sock:
    tcp_sock.bind( (HOST, PORT) )
    tcp_sock.listen()
    conn, addr = tcp_sock.accept()
    with conn:
        print( "Info client conectat (IP Client: ", addr[0], ", Port client: ", addr[1], ")" )
        
        data = conn.recv( BUFFER_SIZE )
        
        # Convertire date primite in array
        image_array = np.frombuffer( data, np.uint8 )
        # Decodare imagine in format OpenCV
        image = cv2.imdecode( image_array, cv2.IMREAD_COLOR )
        
        # Afisare imagine pe ecran intr-o fereastra numita img
        cv2.imshow('img', image)
        # Asteptare apasare tasta
        k = cv2.waitKey(0)
        if( k == ord('q') ):
            print('A fost apasata tasta q')
             # Distrugerea tuturor ferestrelor desenate pe ecran
            cv2.destroyAllWindows()
        
            # Construire si trimitere mesaj inapoi catre client
            message = "stop"
            message_bytes = str.encode( message )
            conn.sendall( message_bytes )
        else:
            print('A fost apasata alta tasta')
        
            # Distrugerea tuturor ferestrelor desenate pe ecran
            cv2.destroyAllWindows()
        
            # Construire si trimitere mesaj inapoi catre client
            message = "primit"
            message_bytes = str.encode( message )
            conn.sendall( message_bytes )

    tcp_sock.close()

Info client conectat (IP Client:  127.0.0.1 , Port client:  56116 )
A fost apasata tasta q


## De rezolvat: 
#### Primiti si afisati imagini de la client pana cand se introduce de la tastatura tasta q.
#### Tasta apasata este citita cu *k = cv2.waitKey(0)*
#### Pentru verificarea tastei apasate, se poate folosi expresia:
`if( k == ord('q') ):
    print('A fost apasata tasta q')
else:
    print('A fost apasata alta tasta')`

In [2]:
import cv2
while(k != ord('q')):
    k = cv2.waitKey(0)
    if( k == ord('q') ):
        print('A fost apasata tasta q')

KeyboardInterrupt: 

# UDP Server

## Exemplu 4: serverul asteapta un mesaj de la client si trimite un raspuns

In [ ]:
import socket

HOST = "127.0.0.1"
PORT = 34512

BUFFER_SIZE = 1024

# Crearea unui socket UDP
with socket.socket( socket.AF_INET, type=socket.SOCK_DGRAM ) as udp_sock:
    # Asignarea unui IP si a unui port pe care serverul sa asculte
    udp_sock.bind( (HOST, PORT) )

    # Ascultare pentru datagrame incoming, de dimensiune maxima BUFFER_SIZE
    data = udp_sock.recvfrom( BUFFER_SIZE )

    message = data[0].decode()
    address = data[1]

    print( "Mesaj de la client: ", message )
    print( "Info client conectat (IP Client: ", address[0], ", Port client: ", address[1], ")" )
    
    # Spargere string provenit de la client si extragere numarului clientului
    client_number = message.split(" ")[-1]

    server_message = "Salutari de la server, client cu numarul " + str( client_number )
    server_message_bytes = str.encode( server_message )

    # Trimitere raspuns catre client
    udp_sock.sendto( server_message_bytes, address )
    
    udp_sock.close()

## De rezolvat: 
#### Pe baza codului de mai sus, scrieti un server UDP care sa astepte la infinit mesaje de la un client, sa numere cate conexiuni au fost deschise si sa ii trimita clientului inapoi indexul socketului pe care l-a deschis